In [9]:
import os
import cv2
import numpy as np
import torch
from PIL import Image
from datetime import datetime
from facenet_pytorch import MTCNN, InceptionResnetV1

# ---------------- CONFIG ----------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DB_FILE = "faces_db.npz"
MATCH_THRESHOLD = 0.45  # cosine similarity threshold
FRAME_SKIP = 2  # process every N-th frame
print("[INFO] Using device:", DEVICE)

# ---------------- MODELS ----------------
mtcnn = MTCNN(image_size=160, margin=20, keep_all=False, device=DEVICE)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(DEVICE)

# ---------------- DATABASE FUNCTIONS ----------------
def load_db(db_path=DB_FILE):
    if os.path.exists(db_path):
        d = np.load(db_path, allow_pickle=True)
        names = d['names'].tolist()
        embeddings = d['embeddings']
        print(f"[DB Loaded] {len(names)} persons")
        return names, embeddings
    print("[DB Loaded] Empty database")
    return [], np.zeros((0, 512), dtype=np.float32)

def save_db(names, embeddings, db_path=DB_FILE):
    np.savez_compressed(db_path, names=np.array(names), embeddings=np.array(embeddings))
    print(f"[DB Saved] {len(names)} persons")

def normalize_embedding(emb):
    return emb / np.linalg.norm(emb)

# ---------------- ENROLLMENT ----------------
def enroll_from_folder(person_name, folder_path, names, embeddings):
    img_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not img_files:
        print(f"[!] No images found for {person_name}")
        return names, embeddings

    person_embs = []
    for img_file in img_files:
        img_path = os.path.join(folder_path, img_file)
        img = Image.open(img_path).convert('RGB')
        face = mtcnn(img)
        if face is None:
            print(f"[!] No face in {img_file}")
            continue
        emb = resnet(face.unsqueeze(0).to(DEVICE)).detach().cpu().numpy()[0]
        emb = normalize_embedding(emb)
        person_embs.append(emb)
        print(f"[+] Enrolled {person_name} from {img_file}")

    if person_embs:
        avg_emb = np.mean(person_embs, axis=0)
        avg_emb = normalize_embedding(avg_emb)
        names.append(person_name)
        embeddings = np.vstack([embeddings, avg_emb])
    return names, embeddings

# ---------------- MATCHING ----------------
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def recognize_face(face_img, names, embeddings):
    face = mtcnn(face_img)
    if face is None:
        return "Unknown", None
    emb = resnet(face.unsqueeze(0).to(DEVICE)).detach().cpu().numpy()[0]
    emb = normalize_embedding(emb)
    sims = [cosine_similarity(emb, e) for e in embeddings]
    if not sims:
        return "Unknown", None
    best_idx = int(np.argmax(sims))
    if sims[best_idx] >= MATCH_THRESHOLD:
        return names[best_idx], sims[best_idx]
    else:
        return "Unknown", sims[best_idx]

# ---------------- ATTENDANCE ----------------
def mark_attendance(name, log_file="attendance.csv"):
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M:%S")

    if not os.path.exists(log_file):
        with open(log_file, "w") as f:
            f.write("date,name,time\n")

    with open(log_file, "r") as f:
        if f"{date_str},{name}" in f.read():
            return  # already marked today

    with open(log_file, "a") as f:
        f.write(f"{date_str},{name},{time_str}\n")
    print(f"[Attendance] {name} at {time_str}")

# ---------------- REAL-TIME RECOGNITION ----------------
def realtime_recognition(names, embeddings, cam_id=0):
    cap = cv2.VideoCapture(cam_id)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % FRAME_SKIP != 0:
            cv2.imshow("Face Attendance", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb_frame)

        name, sim = recognize_face(pil_img, names, embeddings)

        if name != "Unknown":
            mark_attendance(name)
            label = f"{name} ({sim:.2f})"
            color = (0, 255, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow("Face Attendance", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# ---------------- MAIN ----------------
if __name__ == "__main__":
    # Step 1: Load DB
    names, embeddings = load_db()

    # Step 2: Enroll new people (folders: dataset/PersonName)
    names, embeddings = enroll_from_folder("Absar", "dataset/Absar", names, embeddings)
    names, embeddings = enroll_from_folder("Mujahid", "dataset/Mujahid", names, embeddings)
    names, embeddings = enroll_from_folder("Sikandar", "dataset/Sikandar", names, embeddings)

    # Step 3: Save DB
    save_db(names, embeddings)

    # Step 4: Real-time recognition
    realtime_recognition(names, embeddings)


[INFO] Using device: cuda
[DB Loaded] 8 persons
[+] Enrolled Absar from absar.png
[+] Enrolled Mujahid from mujahid.png
[+] Enrolled Sikandar from IMG_1640.JPG
[+] Enrolled Sikandar from sikandar1.jpg
[+] Enrolled Sikandar from sk.jpg
[DB Saved] 11 persons
[Attendance] Mujahid at 15:12:34
[Attendance] Absar at 15:12:54


In [12]:
import os
import cv2
import numpy as np
import torch
import pandas as pd
from PIL import Image
from datetime import datetime
from facenet_pytorch import MTCNN, InceptionResnetV1

# ---------------- CONFIG ----------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DB_FILE = "faces_db.npz"
ATT_FILE = "attendance.xlsx"
MATCH_THRESHOLD = 0.70  # cosine similarity threshold
FRAME_SKIP = 2  # process every N-th frame
print("[INFO] Using device:", DEVICE)

# ---------------- MODELS ----------------
mtcnn = MTCNN(image_size=160, margin=20, keep_all=True, device=DEVICE)  # keep_all=True for multiple faces
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(DEVICE)

# ---------------- DATABASE FUNCTIONS ----------------
def load_db(db_path=DB_FILE):
    if os.path.exists(db_path):
        d = np.load(db_path, allow_pickle=True)
        names = d['names'].tolist()
        embeddings = d['embeddings']
        print(f"[DB Loaded] {len(names)} persons")
        return names, embeddings
    print("[DB Loaded] Empty database")
    return [], np.zeros((0, 512), dtype=np.float32)

def save_db(names, embeddings, db_path=DB_FILE):
    np.savez_compressed(db_path, names=np.array(names), embeddings=np.array(embeddings))
    print(f"[DB Saved] {len(names)} persons")

def normalize_embedding(emb):
    return emb / np.linalg.norm(emb)

# ---------------- ENROLLMENT ----------------
def enroll_from_folder(person_name, folder_path, names, embeddings):
    img_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not img_files:
        print(f"[!] No images found for {person_name}")
        return names, embeddings

    person_embs = []
    for img_file in img_files:
        img_path = os.path.join(folder_path, img_file)
        img = Image.open(img_path).convert('RGB')
        face = mtcnn(img)
        if face is None or (isinstance(face, list) and len(face) == 0):
            print(f"[!] No face in {img_file}")
            continue
        if isinstance(face, list):  # multiple faces returned
            face = face[0]

        if face.ndim == 3:
            face = face.unsqueeze(0)

        emb = resnet(face.to(DEVICE)).detach().cpu().numpy()[0]
        emb = normalize_embedding(emb)
        person_embs.append(emb)
        print(f"[+] Enrolled {person_name} from {img_file}")

    if person_embs:
        avg_emb = np.mean(person_embs, axis=0)
        avg_emb = normalize_embedding(avg_emb)
        names.append(person_name)
        embeddings = np.vstack([embeddings, avg_emb])
    return names, embeddings

# ---------------- MATCHING ----------------
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def recognize_faces(face_tensors, names, embeddings):
    recognized = []
    if embeddings.shape[0] == 0:
        return [("Unknown", None)] * len(face_tensors)

    for face_tensor in face_tensors:
        if face_tensor is None:
            recognized.append(("Unknown", None))
            continue

        if face_tensor.ndim == 3:
            face_tensor = face_tensor.unsqueeze(0)

        emb = resnet(face_tensor.to(DEVICE)).detach().cpu().numpy()[0]
        emb = normalize_embedding(emb)

        sims = [cosine_similarity(emb, e) for e in embeddings]
        best_idx = int(np.argmax(sims))
        if sims[best_idx] >= MATCH_THRESHOLD:
            recognized.append((names[best_idx], sims[best_idx]))
        else:
            recognized.append(("Unknown", sims[best_idx]))
    return recognized

# ---------------- ATTENDANCE ----------------
def init_attendance(names, att_file=ATT_FILE):
    today = datetime.now().strftime("%Y-%m-%d")
    if os.path.exists(att_file):
        df = pd.read_excel(att_file, index_col=0)
    else:
        df = pd.DataFrame(index=names)

    if today not in df.columns:
        df[today] = "A"
    df.to_excel(att_file)
    return df

def mark_attendance(name, att_file=ATT_FILE):
    today = datetime.now().strftime("%Y-%m-%d")
    df = pd.read_excel(att_file, index_col=0)

    # Ensure today's column exists
    if today not in df.columns:
        df[today] = "A"   # default Absent for all students

    # Mark attendance only if not already marked
    if name in df.index:
        current_status = str(df.at[name, today])  # force single cell string
        if current_status != "P":
            df.at[name, today] = "P"
            df.to_excel(att_file)
            print(f"[Attendance] {name} marked Present")
        else:
            print(f"[Attendance] {name} already marked Present")


# ---------------- REAL-TIME RECOGNITION ----------------
def realtime_recognition(names, embeddings, cam_id=0):
    cap = cv2.VideoCapture(cam_id)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % FRAME_SKIP != 0:
            cv2.imshow("Face Attendance", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb_frame)

        # Detect faces + boxes
        boxes, probs = mtcnn.detect(pil_img)
        faces = mtcnn(pil_img)

        if boxes is not None and faces is not None:
            if isinstance(faces, torch.Tensor) and faces.ndim == 3:
                faces = faces.unsqueeze(0)

            results = recognize_faces(faces, names, embeddings)

            for box, (name, sim) in zip(boxes, results):
                x1, y1, x2, y2 = [int(b) for b in box]
                if name != "Unknown":
                    mark_attendance(name)
                    label = f"{name} ({sim:.2f})"
                    color = (0, 255, 0)
                else:
                    label = "Unknown"
                    color = (0, 0, 255)

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("Face Attendance", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



[INFO] Using device: cuda


In [13]:


# ---------------- MAIN ----------------
if __name__ == "__main__":
    # Step 1: Load DB
    names, embeddings = load_db()

    # # Step 2: Enroll new people (folders: dataset/PersonName)
    # names, embeddings = enroll_from_folder("Absar", "dataset/Absar", names, embeddings)
    # names, embeddings = enroll_from_folder("Mujahid", "dataset/Mujahid", names, embeddings)
    # names, embeddings = enroll_from_folder("Sikandar", "dataset/Sikandar", names, embeddings)

    # # Step 3: Save DB
    # save_db(names, embeddings)

    # Step 4: Init attendance Excel
    init_attendance(names)

    # Step 5: Real-time recognition
    realtime_recognition(names, embeddings)

[DB Loaded] 3 persons
[Attendance] Sikandar marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Present
[Attendance] Sikandar already marked Pres